In [1]:
from sage.symbolic.expression import Expression
from delierium.MatrixOrder import Context
from IPython.display import Math
from dataclasses import dataclass
from delierium.helpers import latexer
from sage.symbolic.expression_conversions import ExpressionTreeWalker
from IPython.core.debugger import set_trace

In [2]:
x, y= var("x y")
xi  = function("xi")(x,y)
eta = function("eta")(x,y)
ctx = Context([xi, eta], [x,y])
xi_x = diff(xi, x)
xi_y = diff(xi, y)
xi_xx = diff(xi, x,x)
xi_yy = diff(xi, y,y)
xi_xy = diff(xi, x,y)
eta_x = diff(eta, x)
eta_y = diff(eta, y)
eta_xx = diff(eta, x,x)
eta_yy = diff(eta, y,y)
eta_xy = diff(eta, x,y)

A = function("A")(x,y)
B = function("B")(x,y)
C = function("C")(x,y)
D = function("D")(x,y)
A_x = diff(A, x)
A_y = diff(A, y)
B_x = diff(B, x)
B_y = diff(B, y)
C_x = diff(C, x)
C_y = diff(C, y)
D_x = diff(D, x)
D_y = diff(D, y)

In [3]:
e1 = xi_yy - 2*A*eta_y - B*xi_y + A*xi_x - A_y*eta-A_x*xi
e2 = eta_xx - D*eta_y +C*eta_x +2*D*xi_x +D_y*eta+D_x*xi
e3 = eta_xy - xi_xx/2 +B*eta_x+3*D*xi_y/2+C*xi_x/2 + C_y*eta/2 + C_x*xi/2
e4 = eta_yy - 2*xi_xy +B*eta_y + 3*A*eta_x + 2*C*xi_y + B_y*eta +B_x*xi

In [4]:
from functools import partial, wraps, cache
from operator import mul
#def contextualize(func):
#    @wraps(function)
#    def context_wrapper(context, *args):
#        return partial(func, context)
#    return context_wrapper

def contexter(obj, func=None):
    if func is None:
        return partial(func, obj)
    setattr(obj, func.__name__, func)
    return func

In [5]:
_Sage_Expression = sage.symbolic.expression.Expression
_Delierium_Context = Context
def gen_summands(context: _Delierium_Context, expression: _Sage_Expression) -> list[_Sage_Expression]:
    try :
        if expression.operator().__name__ == 'add_vararg':
            return expression.operands()
    except AttributeError:
        return [expression]
    return [expression]

#all_delierium_functions = [gen_dt_list]



#def curry_funcs_with_context(context):
#    for v in all_delierium_functions:
#        print(id(v))
#        v = partial(v, context)
#        print(id(v))

#curry_funcs_with_context(ctx)
gen_summands = partial(gen_summands, ctx)
#gen_dt = partial(gen_dt, ctx)

In [6]:
print(gen_summands)

functools.partial(<function gen_summands at 0x7f00fee43f60>, <delierium.MatrixOrder.Context object at 0x7f00fccdb240>)


In [7]:
gen_summands(e1)

[-xi(x, y)*diff(A(x, y), x),
 -eta(x, y)*diff(A(x, y), y),
 -2*A(x, y)*diff(eta(x, y), y),
 A(x, y)*diff(xi(x, y), x),
 -B(x, y)*diff(xi(x, y), y),
 diff(xi(x, y), y, y)]

In [8]:
class DelieriumNotALinearPDE(TypeError):
    def __init__ (self, expression):
        #self.message = f"{expression=} is not a valid linear PDE"
        super().__init__(f"{expression=} is not a valid term in linear PDE")

from functools import cache

@dataclass
class DT:
    context: Context
    expression: _Sage_Expression
    coefficient: _Sage_Expression
    derivative: _Sage_Expression
    function: _Sage_Expression
    order: tuple[int]
    comparison_order: tuple[int]
    def __lt__ (self, other):
        r = self.context._weight * (vector(self.comparison_order) - vector(other.comparison_order))
        for entry in r:
            if entry:
                return entry > 0
        return False


In [9]:
def is_derivative(e):
    try:
        return e.operator().__class__.__name__ == "FDerivativeOperator"
    except AttributeError:
        return False

def order_of_derivative(context, e, required_len=0):
    '''Returns the vector of the orders of a derivative respect to its variables
    [2, 1, 3]
    '''
    res = [0] * len(e.variables())
    if not is_derivative(e):
        return res
    opr = e.operator().parameter_set()
    for variable in e.variables():
        i = context._independent.index(variable)
        res[i] = opr.count(i)
    return res


def _compute_comparison_vector(context, func):
    iv = [0] * len(context._dependent)
    if func in context._dependent:
        iv[context._dependent.index(func)] += 1
    return iv

def compute_order(context, expression):
    """computes the monomial tuple from the derivative part"""
    if is_derivative(expression):
        return order_of_derivative(expression, context, len(context._independent))
    # XXX: Check can that be within a system of linead PDEs ? I think not, as the function must then be identically to zero
    return [0] * len(context._independent), None

In [16]:
def _gen_dt(context: _Delierium_Context, expression: _Sage_Expression) -> bool:
    coeff = []
    derivs = []
    match expression.operator().__class__.__name__:
        case "FDerivativeOperator":
            if expression.operator().function() in context._dependent:
                derivs.append(expression)
            else:
                coeff.append(expression)
        case "Expression":
            print("Symbolic Expression")
            print(dir(expression))
        case "NewSymbolicFunction":
            if expression.operator() in context._dependent:
                derivs.append(expression)
            else:
                coeff.append(expression)
        case "function":
            if expression.operator().__name__ == "mul_vararg":
                for operand in expression.operands():
                    if not operand.operator():
                        coeff.append(operand)
                    else:
                        c, d = _gen_dt(context, operand)
                        coeff.extend(c)
                        derivs.extend(d)
        case _:
            print("MISTTT")
    if len(derivs) > 1:
        print(f"{expression=} seems not be part of a linear PDE")
        print(f"{locals()=}")
    return coeff, derivs

def gen_dt(context: _Delierium_Context, expression: _Sage_Expression)-> bool:
    c, d = _gen_dt(context, expression)
    if not d:
        raise DelieriumNotALinearPDE(expression)
    d = reduce(mul, d, 1)
    order = order_of_derivative(context, d)
    return DT(context,
              expression,
              reduce(mul, c, 1), d,
              d.operator().function() if is_derivative(d) else d.function(),              
              order,
              _compute_comparison_vector(context, d.operator().function() if is_derivative(d) else d.function()) + order
             )
              
    

In [17]:
gen_dt(ctx, 2*A_x*B_x*xi_xy)
print("1"*20)
gen_dt(ctx, xi)
print("2"*20)
gen_dt(ctx, 2*xi)
#print("3"*20)
#gen_dt(ctx, A)
#print("4"*20)
#gen_dt(ctx, 2*A*D)
print("5"*20)
gen_dt(ctx, xi_x)


11111111111111111111
22222222222222222222
55555555555555555555


DT(context=<delierium.MatrixOrder.Context object at 0x7f00fccdb240>, expression=diff(xi(x, y), x), coefficient=1, derivative=diff(xi(x, y), x), function=xi, order=[1, 0], comparison_order=[1, 0, 1, 0])

In [18]:
gen_dt(ctx, 2*A_x*B_x*xi_xy)

DT(context=<delierium.MatrixOrder.Context object at 0x7f00fccdb240>, expression=2*diff(A(x, y), x)*diff(B(x, y), x)*diff(xi(x, y), x, y), coefficient=2*diff(A(x, y), x)*diff(B(x, y), x), derivative=diff(xi(x, y), x, y), function=xi, order=[1, 1], comparison_order=[1, 0, 1, 1])

In [19]:
v = list(map(lambda _: gen_dt(ctx, _), gen_summands(e1)))

In [20]:
from pprint import pprint
e1.show()
pprint(v)

-xi(x, y)*diff(A(x, y), x) - eta(x, y)*diff(A(x, y), y) - 2*A(x, y)*diff(eta(x, y), y) + A(x, y)*diff(xi(x, y), x) - B(x, y)*diff(xi(x, y), y) + diff(xi(x, y), y, y)

[DT(context=<delierium.MatrixOrder.Context object at 0x7f00fccdb240>,
    expression=-xi(x, y)*diff(A(x, y), x),
    coefficient=-diff(A(x, y), x),
    derivative=xi(x, y),
    function=() |--> xi(x, y),
    order=[0, 0],
    comparison_order=[0, 0, 0, 0]),
 DT(context=<delierium.MatrixOrder.Context object at 0x7f00fccdb240>,
    expression=-eta(x, y)*diff(A(x, y), y),
    coefficient=-diff(A(x, y), y),
    derivative=eta(x, y),
    function=() |--> eta(x, y),
    order=[0, 0],
    comparison_order=[0, 0, 0, 0]),
 DT(context=<delierium.MatrixOrder.Context object at 0x7f00fccdb240>,
    expression=-2*A(x, y)*diff(eta(x, y), y),
    coefficient=-2*A(x, y),
    derivative=diff(eta(x, y), y),
    function=eta,
    order=[0, 1],
    comparison_order=[0, 1, 0, 1]),
 DT(context=<delierium.MatrixOrder.Context object at 0x7f00fccdb240>,
    expression=A(x, y)*diff(xi(x, y), x),
    coefficient=A(x, y),
    derivative=diff(xi(x, y), x),
    function=xi,
    order=[1, 0],
    comparison_order=[1,

In [21]:
class DP:
    def __init__(self, context, expression):
        self.context = context
        self.expression = expression
        self.dts = list(sorted(map(lambda _: gen_dt(context, _), gen_summands(expression))))
        lcoeff = self.dts[0].coefficient
        for dt in self.dts:
            dt.coefficient /= lcoeff
        self.lder = self.dts[0].derivative
        


In [22]:
dp=DP(ctx, e1)

In [23]:
dp.lder

diff(xi(x, y), x)

In [24]:
for _ in dp.dts:
    print(_.coefficient, _.derivative, _.comparison_order)


1 diff(xi(x, y), x) [1, 0, 1, 0]
1/A(x, y) diff(xi(x, y), y, y) [1, 0, 0, 2]
-B(x, y)/A(x, y) diff(xi(x, y), y) [1, 0, 0, 1]
-2 diff(eta(x, y), y) [0, 1, 0, 1]
-diff(A(x, y), x)/A(x, y) xi(x, y) [0, 0, 0, 0]
-diff(A(x, y), y)/A(x, y) eta(x, y) [0, 0, 0, 0]
